In [ ]:
# Packages

from Bio.Phylo.TreeConstruction import DistanceCalculator

import numpy as np
from scipy.spatial.distance import pdist
from sklearn.manifold import TSNE

In [ ]:
# Creation of a dictionnary in which the key will be the id and the value will be the sequence

with open("/content/drive/MyDrive/long_project/premier_alignement.fasta") as out_file:
  for fasta in out_file:
    print(fasta)

>12CEB4764SAL

CACACGTGGAGGCCGACCCCGGCCAGCCCCGGCCTCCCAAGTCATCGGCAGCGCGGGTCGCGGCACGCAAGCGTCCGGGCGTTCTACTGGAGGGGGCCGTCGCGAACTCACAGGTTGGCCTGTGGCGCCAGCGAGGGCCGGTGGCAGGACCCCGGGAAGGGGCCGTGCCGTCGCGGCTGAGGAGCGCGGGCGATAGCCCCGGCCGTCCCCCAGTTCCCCCGGGCTCTGGCCGCCGACCGCCGGGTCGCCGGCTGCCCATTCCGCACTGGGCCGTCGCCCAAGGTCGCCGCCGCGCGGACACCCCCGGCTTATGCAGAGCCCCGCCGCCGACCCGTCTCTTGCCTGCCCGGGCGACCCGCTGACTCAATCTAAGCGCGTAACCCGCTGTTGTTACTGGCCAAAATCGCCCCGCGGGCTGCCCCGCCCCGGTTCCCCGTTCATCACCCTTTCGGCTACTGGGGTCGACGGTCCGTTTAGCGCATCTGCTGCCGGCCCACCCAGTGCGCCCGATCGAGGCCCAGGTCCGGGCGCCCTCCGCCGCACCTCCGACAGGACGTCAACGCCCCGCGCCGTGCGGCCTGCCGAGGGCTAGGTACGCTCTGGGCCACCCACCTCCCCCGTGGCGTCCTACCCTCGCACGGGCCCCGCCCACCAACCATGTGCCTACCCGCCTACAGCGCGGTTCCCCCAGAGCCCATGCTATCTAGCCCTCAGCGACGCGGGATGCACCCGCGACTGCGCATGGCCCGAGGCGGACTCCATGAGCTCGCGGGACGGCTGCCGCCGGGCCCCCGCGTCACCCTCTCAACCTTGACACGGGCGAATATGCCTCGCCCCTGGACGGCGGTCTGGAAGTGTCCCCCCCGACCCTTCACTATTAACCGCGCCCTTCACGACGCGCGCGCCCCCCAGGAGCGAGCGGCCCGACCGACCGTGGCCGCCGACAGCATCTGCTAGTCATCCAGGAACGGTATGCGGGAGGCCG

In [ ]:
# Calculation of distance matrix

test = np.array([['A','T','C','G'], 
                 ['A','T','C','C'],
                 ['T','A','G','A']])

In [ ]:
# First we will try to create the t-SNE keeping the same dimensions (=?)

# It would be possible to reduce the number of dimensions to a reasonable amount (50?) using another
# dimensionality reduction method (PCA for dense data or truncatedSVD for sparse data).

In [ ]:
test = TSNE(n_components=2,
            perplexity=30,
            early_exaggeration=12,
            learning_rate=200,
            n_iter=1000,
            n_iter_without_progress=300,
            min_grad_norm=1e-7
            metric=

            )